In [ ]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *

In [ ]:
plates = [
    # '478_20230814', '468_20230809', '447_20230821', '487_20230922',
    #    '492_20230901', '471_20230821',
    # '486_20231009',
    "494_20230908",
    #       '482_20230908',
    #       '495_20231013',
    #       '463_20231013',
    #    '491_20231013',
    #       '481_20231005', '483_20231005',
    # '490_20231003',
    # '485_20230929',
    # "310_20230830"
]
# plates = ["3_20220426", "12_20220502", "16_20220419", "480_20221205"]
# directory_targ = directory_project
directory_targ = os.path.join(directory_scratch, "stitch_temp") + "/"
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
# update_plate_info(directory_targ, local=True)
# all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

select = analysis_folders
select["total_path"] = select["total_path"].str.replace(
    "/projects/0/einf914/data/", "/scratch-shared/amftrack/stitch_temp/"
)

In [ ]:
analysis_folders

In [ ]:
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
    load_skel,
)

run_info = whole_plate_info
t = 0
select = run_info.loc[run_info["t"] == t]
row = [row for index, row in select.iterrows()][0]
path_exp = f'{directory}{row["path_exp"]}'
exp = pickle.load(open(path_exp, "rb"))
exp.save_location = "/".join(path_exp.split("/")[:-1])
exp.directory = directory
load_study_zone(exp)
load_graphs(exp, directory, indexes=[t], post_process=True)

In [ ]:
for t in range(exp.ts):
    exp.load_tile_information(t)

In [ ]:
get_length_density_in_region(exp, t, {"i": 0})

In [ ]:
exp.image_paths

In [ ]:
time = "6:40:00"
directory = directory_targ
max_ind = 20
incr = 100

list_f = [
    get_num_trunks,
    get_area,
    get_area_separate_connected_components,
    get_num_tips,
    get_num_nodes,
    get_area_study_zone,
    get_num_tips_study_zone,
    get_num_nodes_study_zone,
    get_num_edges,
    get_length_tot,
    get_length_study_zone,
    get_is_out_study,
    get_mean_edge_straight,
    get_spore_volume,
    get_num_spores,
    get_tot_biovolume_study,
    get_tot_biovolume,
]
list_args = [{}] * len(list_f)
overwrite = True
num_parallel = 6
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    select,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
)

In [ ]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.P_regions import *

%load_ext autoreload
%autoreload 2
directory = directory_targ
overwrite = True
load_graphs = True
name_job = "density3"
# list_f = [
#     get_length_tot,
# ]
# list_args = [{}] * len(list_f)
max_ind = 20
incr = 100
num_parallel = 15
time = "1:00:00"
list_f = [get_surface_area_in_region] * 18
list_f += [get_length_density_in_region] * 18
list_args = [{"i": i} for i in range(18)] * 2
# fs = [
#     get_density_in_ring_new,
#     get_density_active_tips_in_ring,
# ]

# list_f = []
# list_args = []

# for f in fs:
#     list_f += [f] * 20

#     list_args += [{"incr": incr, "i": i, "rh_only": True,"max_t" : 100} for i in range(0, 20)]
args = [directory, True, True]
directory_targ = directory
for index, row in select.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        whole_plate_info = whole_plate_info.loc[whole_plate_info["t"].between(0, 200)]

        run_parallel_post(
            "time_plate_post_process_long.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "density_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
            # dependency = True,
        )

In [ ]:
1

In [ ]:
name_job = "density3"
run_launcher(
    "analysis_uploader_no_upload.py",
    [directory_targ, name_job, -1],
    plates,
    "6:00:00",
    dependency=True,
    name_job=name_job,
)

In [ ]:
list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    # get_width_tip_edge,
    # get_width_root_edge,
    # get_width_average,
    get_has_reached_final_pos,
    get_in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )